# Criando dataset: dados programas de Pós-graducação UFMG

Este notebook tem como finalidade processar, unificar e transpor dados referentes aos programas de Pós-graduação da UFMG, tendo como fonte os dados abertos fornecidos pela plataforma Sucupira.<br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### Função: read_csv
***Method***: Tem como objetivo ler um arquivo csv e transformá-lo em um dataFrame.<br><br>
***Paramns***: Funcao recebe como parametro um caminho como o nome do arquivo(***path***), um separador de colunas (***separator***) e um encoding opcional (***encode_type***).<br><br>
***Return***: Funcao tem como retorno um dataFrame.<br>


In [ ]:
def read_dataFrame(path, separator=';', encode_type='ISO-8859-1'):
    alternate_sep = ','
    try:
        return pd.read_csv(path, engine='c', sep=separator, encoding=encode_type)
    except Exception as ex:
        # A partir de um ano ai eles resolveram: vamo mudar pra ,
        #print(ex)
        return pd.read_csv(path, engine='c', sep=alternate_sep, encoding=encode_type)

### Função: merge_data_frame
***Method***: Tem como objetivo unir todos os arquivos CSV de uma determinada categória<br><br>
***Paramns***: Funcao recebe como parametro: 

    * um caminho como o nome do arquivo(***path_type***),
    * um separador de colunas (***slice_data***)
    * um encoding opcional (***list_columns***).
    * um encoding opcional (***list_new_columns***).
    * um encoding opcional (***drop_dup***).

***Return***: Funcao tem como retorno um dataFrame.<br>


In [ ]:
def merge_data_frame(path_type, slice_data, list_columns, list_new_columns, drop_dup = False, drop_nan=False, encode_type="ISO-8859-1"):

    # default do drop dup e false para evitar que ele ignore o dataset das teses
    
    # Define directory 
    path_year_list = os.listdir(path_type)

    # Empty data frame
    df_merge = pd.DataFrame({})

    # Procces list of file in currente year
    for file in path_year_list:
        df = None
            
        # Read a csv file
        df = read_dataFrame(path_type + file, encode_type=encode_type)

        # Query for UFMG
        df = df.query(slice_data)

        # Slice data frame
        df = df[list_columns]
        
        df.columns = list_new_columns
        
        # Checa se ele vai dropar as linhas duplicadas
        if drop_dup == True:
            df = df.drop_duplicates()
        
        # Checa se ele vai remover linhas com entradas invalidas
        if drop_nan == True:
            df = df.dropna()
        
        # Concat data frame
        df_merge = pd.concat([df_merge,df])
            
    return df_merge

### Função: mapDataFrame
***Method***: Tem como objetivo realizar um mapeamento no data frame<br><br>
***Paramns***: Funcao recebe como parametro: 

    * um caminho como o nome do arquivo(***path_type***),
    * um separador de colunas (***slice_data***)
    * um encoding opcional (***list_columns***).
    * um encoding opcional (***list_new_columns***).
    * um encoding opcional (***drop_dup***).

***Return***: Funcao tem como retorno um dataFrame.<br>


In [ ]:
def mapDataFrame(df, groupList, columnsName, listAgg=None, dropColList=None):
    
    # Dimensao do dataframe
    n = df.shape[0]
    mapValues = {}

    for i in range(n):
        # Define uma chave para mapeamento do dataframe
        key = ''
        for val in groupList:
            try:
                key = key +""+str(int(df.iloc[i,val]))
            except:
                key = key +""+str(df.iloc[i,val])
        
        # Checa se a chave ja existe
        if key in mapValues:        
            if listAgg is None:
                # Adiciona no final da lista o atributo de contagem de ocorrencias
                mapValues[key][-1] += 1
            else:
                # Define tamanho do slice
                l = len(listAgg)*-1
                
                for j in range(len(listAgg)):
                    # Indice da coluna a ser somando
                    k = listAgg[j]

                    # Valor na posicao desejada
                    contVal = df.iloc[i, l]

                    # Soma coluna desejada
                    mapValues[key][l] =  mapValues[key][l] + contVal

                    # Incrementa indice de slice
                    l += 1
        else:
            # Recebe lista de valores para a linhda do dataFrame
            listVal = list(df.iloc[i,:].values)
            
            # Adiciona key como valor
            listVal.append(key)
            
            if listAgg is None:
                # Adiciona no final da lista o atributo de contagem de ocorrencias
                listVal.append(1)
            else:
                for j in range(len(listAgg)):
                    # Indice da coluna a ser somando
                    k = listAgg[j]
                    listVal.append(df.iloc[i, k])
            
            mapValues.update({key: listVal})
    
    df = pd.DataFrame(data=mapValues).T

    # Verifica se a opcao de remover colunas foi requisitada
    if dropColList:
        df = df.drop(df.columns[listAgg], axis = 1)
        
    df.columns = columnsName
    
    return df

### Processando teses e dissetações dos programas de Pós-graduação
***Fonte:*** https://dadosabertos.capes.gov.br/group/banco

In [ ]:
# Define empty data frame
df_merge = pd.DataFrame({})

In [ ]:
# 1987-2012  - Teses e Dissertacoes
path_type = "../capes/teses_dissertacoes_programa/1987-2012/"
slice_data = "SiglaIes == 'UFMG'"
list_columns = ['Nivel', 'IdPrograma','NomePrograma', 'AnoBase']
list_new_columns = ['Nivel', 'CodigoPrograma','NomePrograma', 'AnoBase']

# Le as dataframes da pasta e da merge dataframes
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns)

# Remove Mestrados Profissionais e profissionalizantes
df = df[df.Nivel != 'MESTRADO PROFISSIONAL']
df = df[df.Nivel != 'Profissionalizante']

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,3]

# Contando mestrado
df_mestrado = mapDataFrame(df.query("Nivel == 'Mestrado'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Dissertacao"])

# Contando mestrado
df_doutorado = mapDataFrame(df.query("Nivel == 'Doutorado'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Teses"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_doutorado, df_mestrado, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
# 2013-2016 - Teses e Dissertacoes
path_type = "../capes/teses_dissertacoes_programa/2013-2016/"
slice_data = "SG_ENTIDADE_ENSINO == 'UFMG'"
list_columns = ['NM_GRAU_ACADEMICO', 'CD_PROGRAMA', 'NM_PROGRAMA', 'AN_BASE']
dict_agg = {'NomePrograma':['count']}
list_new_columns = ['Nivel', 'CodigoPrograma','NomePrograma', 'AnoBase']

# Le as dataframes da pasta e da merge dataframes
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns)

# Remove Mestrados Profissionais e profissionalizantes
df = df[df.Nivel != 'MESTRADO PROFISSIONAL']
df = df[df.Nivel != 'Profissionalizante']

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,3]

# Contando mestrado
df_mestrado = mapDataFrame(df.query("Nivel == 'MESTRADO'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Dissertacao"])

# Contando mestrado
df_doutorado = mapDataFrame(df.query("Nivel == 'DOUTORADO'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Teses"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_doutorado, df_mestrado, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
# 2017-2018 - Teses e Dissertacoes
path_type = "../capes/teses_dissertacoes_programa/2017-2018/"
slice_data = "SG_ENTIDADE_ENSINO == 'UFMG'"
list_columns = ['NM_GRAU_ACADEMICO', 'CD_PROGRAMA', 'NM_PROGRAMA', 'AN_BASE']
dict_agg = {'NomePrograma':['count']}
list_new_columns = ['Nivel', 'CodigoPrograma','NomePrograma', 'AnoBase']

# Le os dataframes da pasta e da merge dataframes
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns)

# Remove Mestrados Profissionais e profissionalizantes
df = df[df.Nivel != 'MESTRADO PROFISSIONAL']

df = df[df.Nivel != 'Profissionalizante']

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,3]

# Contando mestrado
df_mestrado = mapDataFrame(df.query("Nivel == 'MESTRADO'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Dissertacao"])

# Contando mestrado
df_doutorado = mapDataFrame(df.query("Nivel == 'DOUTORADO'"), listKey, columnsName=['Nivel', 'CodigoPrograma', 'NomePrograma', 'AnoBase','keyA',"Teses"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_doutorado, df_mestrado, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
# Exporta df para nosso dataset
df_merge.to_excel('./datasets/teses_dissertacao-ufmg.xlsx')
df_merge

### Processando produção dos programas de Pós-graduação
***Fonte:*** https://dadosabertos.capes.gov.br/group/producao-intelectual-da-pos-graduacao

In [ ]:
# Empty data_frame to merge data
df_merge = pd.DataFrame({})

In [ ]:
# Produtividade do programa - 2004 - 2012
path_type = "../capes/produtividade_programa/2004-2012/"
slice_data = "SG_ENTIDADE_ENSINO == 'UFMG'"
list_columns = ['NM_PROGRAMA_IES', 'CD_PROGRAMA_IES','AN_BASE','NM_PRODUCAO','NM_TIPO_PRODUCAO']
dict_agg = {'NomePrograma':['count']}
list_new_columns = ['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao']

# Combina os dataframes das pastas
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns, drop_dup=False)

# Removo duplicatas da coluna nome producao, ou seja, tiro nomes repetidos
df = df.drop_duplicates()

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,2]

# Contando producao Artistica
df_artistica = mapDataFrame(df.query("TipoProducao == 'ARTÍSTICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma', 'AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesArtisticas"])

# Contando producao Bibliografica
df_bibliografica = mapDataFrame(df.query("TipoProducao == 'BIBLIOGRÁFICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesBibliograficas"])

# Contando producao Tecnica
df_tecnica = mapDataFrame(df.query("TipoProducao == 'TÉCNICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesTecnicas"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_artistica, df_bibliografica, on="keyA", how="outer")
df_a = pd.merge(df_a, df_tecnica, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
# Produtividade do programa - 2013 - 2016
path_type = "../capes/produtividade_programa/2013-2016/"
slice_data = "SG_ENTIDADE_ENSINO == 'UFMG'"
list_columns = ['NM_PROGRAMA_IES', 'CD_PROGRAMA_IES','AN_BASE','NM_PRODUCAO','NM_TIPO_PRODUCAO']
dict_agg = {'NomePrograma':['count']}
list_new_columns = ['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao']

# Combina os dataframes das pastas
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns, drop_dup=False)

# Removo duplicatas da coluna nome producao, ou seja, tiro nomes repetidos
df = df.drop_duplicates()

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,2]

# Contando producao Artistica
df_artistica = mapDataFrame(df.query("TipoProducao == 'ARTÍSTICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma', 'AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesArtisticas"])

# Contando producao Bibliografica
df_bibliografica = mapDataFrame(df.query("TipoProducao == 'BIBLIOGRÁFICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesBibliograficas"])

# Contando producao Tecnica
df_tecnica = mapDataFrame(df.query("TipoProducao == 'TÉCNICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesTecnicas"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_artistica, df_bibliografica, on="keyA", how="outer")
df_a = pd.merge(df_a, df_tecnica, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
# Produtividade do programa - 2017 - 2020
path_type = "../capes/produtividade_programa/2017-2020/"
slice_data = "SG_ENTIDADE_ENSINO == 'UFMG'"
list_columns = ['NM_PROGRAMA_IES', 'CD_PROGRAMA_IES','AN_BASE','NM_PRODUCAO','NM_TIPO_PRODUCAO']
dict_agg = {'NomePrograma':['count']}
list_new_columns = ['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao']

# Combina os dataframes das pastas
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns, drop_dup=False)

# Removo duplicatas da coluna nome producao, ou seja, tiro nomes repetidos
df = df.drop_duplicates()

In [ ]:
# Define lista de colunas para formar a chave de mapeamento
listKey = [1,2]

# Contando producao Artistica
df_artistica = mapDataFrame(df.query("TipoProducao == 'ARTÍSTICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma', 'AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesArtisticas"])

# Contando producao Bibliografica
df_bibliografica = mapDataFrame(df.query("TipoProducao == 'BIBLIOGRÁFICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesBibliograficas"])

# Contando producao Tecnica
df_tecnica = mapDataFrame(df.query("TipoProducao == 'TÉCNICA'"), listKey, columnsName=['NomePrograma', 'CodigoPrograma','AnoBase', 'NomeProducao','TipoProducao','keyA',"ProducoesTecnicas"])

# Realiza uniao dos dataFrames por atributo
df_a = pd.merge(df_artistica, df_bibliografica, on="keyA", how="outer")
df_a = pd.merge(df_a, df_tecnica, on="keyA", how="outer")

# Junta os dataframes das teses
df_merge = pd.concat([df_merge,df_a])

In [ ]:
df_merge.to_excel('./datasets/produtividade_do_programa.xlsx')

### Processando dado sobre bolsas 
***Fonte:*** https://geocapes.capes.gov.br/geocapes/

In [ ]:
# Empty data_frame to merge data
df_merge = pd.DataFrame({})

In [ ]:
# Bolsas GeoCapes
path_type = "../capes/geocapes/"
slice_data = "IES == 'UFMG'"
list_columns = ['Ano','Código Programa', 'Programa','DOUTORADO PLENO', 'MESTRADO']
dict_agg = {'Programa':['count']}
list_new_columns = ['AnoBase','CodigoPrograma','Programa','BolsasDoutorado', 'BolsasMestrado']

# Combina os dataframes das pastas
df = merge_data_frame(path_type, slice_data, list_columns, list_new_columns, drop_dup=True, drop_nan=True, encode_type="utf-8")

# Define lista de colunas para formar a chave de mapeamento
listKey = [1,0]

# Define lista de colunas a serem somadas
listSum = [3,4]

# Contando mestrado
df_merge = mapDataFrame(df, listKey, columnsName=['AnoBase','CodigoPrograma','NomePrograma','keyA','BolsasDoutorado', 'BolsasMestrado'], listAgg=listSum, dropColList=True)

In [ ]:
df_merge.to_excel('./datasets/bolsas_programas.xlsx')

### Juntando datasets:
#### Producao, Teses/Dissertações, Bolsas, Valor das Bolsas, Nota Capes

In [ ]:
# Importando resultado para teses
df_producao = pd.read_excel('./datasets/produtividade_do_programa.xlsx')
df_producao.head(10)

In [ ]:
# Importando resultado para teses
df_teses = pd.read_excel('./datasets/teses_dissertacao-ufmg.xlsx')
df_teses.head(10)

In [ ]:
# Importando resultado para teses
df_bolsas = pd.read_excel('./datasets/bolsas_programas.xlsx')
df_bolsas.head(10)

In [ ]:
# Merge dataframes
df_merge = pd.merge(df_producao, df_teses, on="keyA", how="outer")
df_merge = pd.merge(df_merge, df_bolsas, on="keyA", how="outer")
df_merge.head(10)

In [ ]:
df_valores = read_dataFrame('./datasets/valor_bolsas.csv',";")
df_valores.head(10)

In [ ]:
def multVal(df, df_valores):
    
    n = df_valores.shape[0]
    bolsaAno= {}
    
    for i in range(n):
        ano = str(int(df_valores.loc[i,"AnoBase"]))
        bolsaAno.update({ano: list(df_valores.iloc[i,1:])})
    
    n = df.shape[0]
    
    valDoutorado = []
    valMestrado = []
    
    for i in range(n):
        ano = str(int(df.loc[i,"AnoBase_y_x"]))
        mestrado = df.loc[i,"BolsasMestrado"]
        doutorado = df.loc[i,"BolsasDoutorado"]
        
        if ano in bolsaAno:
            anualMestrado = mestrado*bolsaAno[ano][1]*12
            anualDoutorado = doutorado*bolsaAno[ano][0]*12
            
            valDoutorado.append(anualDoutorado)
            valMestrado.append(anualMestrado)
            
        else:
            valDoutorado.append(0)
            valMestrado.append(0)
    
    df = df.assign(valor_anual_bolsa_doutorado = valDoutorado)
    df = df.assign(valor_anual_bolsa_mestrado = valMestrado)
    
    return df
    
df_merge = df_merge.fillna(0)
df_valores = multVal(df_merge, df_valores)
df_valores.head(10)

In [ ]:
# Merge dataframes
df_merge = pd.merge(df_merge, df_valores, on="keyA", how="outer")
df_merge.head(10)

#### Definindo notas Capes para anos correspondentes

In [ ]:


# Abrindando arquivo com notas capes
df_nota = pd.read_excel('./datasets/notas_programas_capes.xlsx')

# Ordena dataFrame    
df_nota = df_nota.sort_values(["keyA","AnoBase"])

a = {}
n = df_nota.shape[0]

# Cria coluna com anos que não houve avaliacao qaudrienal
for i in range(n):
    codigoPrograma = df_nota.iloc[i,2]
    ano = df_nota.iloc[i,0]
    
    if codigoPrograma in a:
        a[codigoPrograma]
        a[codigoPrograma].update({"CodigoPrograma" : codigoPrograma})
        a[codigoPrograma].update({ano : list(df_nota.iloc[i,-2:])})
    else:
        a.update({codigoPrograma : {}})
        a[codigoPrograma].update({"CodigoPrograma" : codigoPrograma})
        a[codigoPrograma].update({ano : list(df_nota.iloc[i,-2:])})
        
        
result = pd.DataFrame(data=a).T 
n, m = result.shape
anos = result.columns

# Transforma valores NaN in 0
result = result.fillna(0)

# Define nota para anos que não houve avaliacao quadrienal
for i in range(n):
    codigoPrograma = result.iloc[i,1]
    for j in  range(m):
        if result.iloc[i,j] == 0:
            result.iloc[i,j] = result.iloc[i,j-1]

b = {}
for i in range(n):
    codigoPrograma = result.iloc[i,0]

    for j in  range(m):
        ano = anos[j]
        val = result.iloc[i,j]
        
        if val[0] == 0:
            val = [0,0]
        
        if codigoPrograma in b:
            rowList = [codigoPrograma,ano,val[0],val[1]]
            b[codigoPrograma].append(rowList)
        else:
            b.update({codigoPrograma: list()})
            rowList = [codigoPrograma, ano, val[0],val[1]]
            b[codigoPrograma].append(rowList)

c = []

# Transforma dicionario em lista    
for key in b.keys():
    for i in range(1,len(b[key])):
        c.append(b[key][i])

df_notas = pd.DataFrame(c, columns =['CodigoPrograma','AnoBase', 'NotaCapesDoutorado','NotaCapesMestrado']) 
df_notas['keyA'] = df_notas['CodigoPrograma'] + df_notas['AnoBase'].astype(str)
df_notas = df_notas.query("AnoBase !='2019'")
df_notas

In [ ]:
# Merge dataframes
df_merge = pd.merge(df_merge, df_notas, on="keyA", how="outer")
df_merge.head(10)

In [ ]:
'''
Função une colunas que tiveram clash nos full joins e coloca elas e retorna uma coluna.
Recebe um data frame, as colunas para serem unidas e o tamanho do dataframe
'''
def recreate_column(df, list_of_cols, shape):
    coluna = ['NaN']* shape
    for index, row in df.iterrows():
        for col in list_of_cols:
            valor = str(row[col]).lower()
            if valor == "0" or valor == "nan" or valor == "0.0":
                continue
            else:
                #print(valor, type(valor))
                coluna[index] = valor
    return coluna

In [ ]:
ano_prog = ['AnoBase', 'AnoBase_x_x', 'AnoBase_x_x_x', 'AnoBase_x_x_y', 'AnoBase_x_y', 'AnoBase_x_y_x', 'AnoBase_x_y_y', 'AnoBase_y_x', 'AnoBase_y_x_x', 'AnoBase_y_x_y', 'AnoBase_y_y', 'AnoBase_y_y_x', 'AnoBase_y_y_y']
codigo_prog = ['CodigoPrograma', 'CodigoPrograma_x_x', 'CodigoPrograma_x_x_x', 'CodigoPrograma_x_x_y', 'CodigoPrograma_x_y', 'CodigoPrograma_x_y_x', 'CodigoPrograma_x_y_y', 'CodigoPrograma_y_x', 'CodigoPrograma_y_x_x', 'CodigoPrograma_y_x_y', 'CodigoPrograma_y_y', 'CodigoPrograma_y_y_x', 'CodigoPrograma_y_y_y']
nome_prog = ['NomePrograma_x_x', 'NomePrograma_x_x_x', 'NomePrograma_x_x_y', 'NomePrograma_x_y', 'NomePrograma_x_y_x', 'NomePrograma_x_y_y', 'NomePrograma_y_x', 'NomePrograma_y_x_x', 'NomePrograma_y_x_y', 'NomePrograma_y_y', 'NomePrograma_y_y_x', 'NomePrograma_y_y_y']

# Recria as colunas
anos = recreate_column(df_merge, ano_prog, df_merge.shape[0])
codigo = recreate_column(df_merge, codigo_prog, df_merge.shape[0])
nome_prog = recreate_column(df_merge, nome_prog, df_merge.shape[0])

# Reinsere as colunas
df_merge['AnoBase'] = anos
df_merge['CodigoPrograma'] = codigo
df_merge['NomePrograma'] = nome_prog

# Ordena as colunas
df_merge = df_merge.sort_index(axis=1)
df_merge

In [ ]:
# Colunas finais
colsToKeep =[
 'keyA',
 'AnoBase',
 'BolsasDoutorado_x',
 'BolsasMestrado_x',
 'CodigoPrograma',
 'Dissertacao_x',
 'NomePrograma',
 'NotaCapesDoutorado',
 'NotaCapesMestrado',
 'ProducoesArtisticas_x',
 'ProducoesBibliograficas_x',
 'ProducoesTecnicas_x',
 'Teses_x',
 'valor_anual_bolsa_doutorado',
 'valor_anual_bolsa_mestrado'
]


df_merge = df_merge.loc[:,colsToKeep]

# Renomeio Colunas
df_merge.columns = [
    'Chave',
    'AnoBase',
    'BolsasDoutorado',
    'BolsasMestrado',
    'CodigoPrograma',
    'Dissertacao',
    'NomePrograma',
    'NotaCapesDoutorado',
    'NotaCapesMestrado',
    'ProducoesArtisticas',
    'ProducoesBibliograficas',
    'ProducoesTecnicas',
    'NumTeses',
    'ValorAnualBolsaDoutorado',
    'ValorAnualBolsaMestrado'
]

df_merge

In [ ]:
# Exporta dataset limpo
df_merge.to_excel('./datasets/dados_posGraduacao_UFMG.xlsx')
df_merge